# NOTEBOOK RAPPORT SLL

L'objectif de ce notebook est de remplir le rapport du Service du Livre et de la Lecture (SLL).

# A - Site internet

## A1 - Site internet

* Site internet 
* Statut site : Site internet propre

La bibliothèque a-t-elle déposé une attestation de conformité de son site web aux exigences du Référentiel général d'accessibilité pour les administrations RGAA) ? **Non**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import datetime

from kiblib.utils.db import DbConn
from kiblib.utils.code2libelle import Code2Libelle
from kiblib.adherent import Adherent
from kiblib.pret import Pret
from kiblib.document import Document

In [2]:
db_conn = DbConn().create_engine()

year = 2024
#date_extraction_usagers = '2023-12-27'
date_extraction_usagers = '2024-12-25'

# C - Accès et installations
## C1 - Accès et installations

|Nombre d'heures d'ouverture hebdomadaires tous publics|54|C101|
|:-----------------------------------------------------|---|----|
|Nombre de jours d'ouverture hebdomadaires|6|C115|
|Nombre de jours d'ouverture annuels|???|C102|
|La bibliothèque est-elle ouverte dimanche ?|Oui|C116|
|Nombre de places assises disponibles |???|C103 | 
| - Avec possibilité de connexions aux ordinateurs |47|C104|
|Nombre de postes informatiques publics avec accès internet|???|C106|
|La bibliothèque met-elle à disposition des usagers empêchés de lire du fait d’un handicap, des postes informatiques équipés d’aides matérielles et logicielles pour accéder aux collections ?|Non|C114|
|Nombre de photocopieurs en accès public| 1|C109|
|Nombre d'imprimantes en accès public|???|C110
|Nombre de tablettes mises à disposition du public|0|C118|
|Nombre de bibliobus|1|C112
|Liseuses mises à la dispsition du public|0|C118|
|Bornes d'écoute|1|C119|
|Nombre de consoles de jeux-vidéos|6|C121|
|Connexion wifi publique|oui|C120|

## C2 - Catalogue

### Notices

In [3]:
query = f"""SELECT COUNT(bm.biblionumber)
    FROM koha{year}.biblio_metadata bm"""
r = pd.read_sql(query, db_conn)
r

,COUNT(bm.biblionumber)
0,236170


### Notices créées dans l'année

In [4]:
query = f"""SELECT COUNT(bm.biblionumber)
    FROM koha{year}.biblio_metadata bm
    JOIN koha{year}.biblio b ON b.biblionumber = bm.biblionumber 
    WHERE YEAR(b.datecreated) = '{year}'"""
r = pd.read_sql(query, db_conn)
r

,COUNT(bm.biblionumber)
0,8861


### Notices créées dans l'année "à la main"

In [5]:
query = f"""SELECT COUNT(bm.biblionumber)
    FROM koha{year}.biblio_metadata bm
    JOIN koha{year}.biblio b ON b.biblionumber = bm.biblionumber 
    WHERE YEAR(b.datecreated) = '{year}'
    AND ExtractValue( bm.metadata, '//datafield[@tag="801"]/subfield[@code="b"]' ) = '595126101'"""
r = pd.read_sql(query, db_conn)
r

,COUNT(bm.biblionumber)
0,828


### Nombre de notices entrées par importation dans l'année

In [6]:
# Nombre de notices entrées dans l'année - nombre de notices entrées à la main
query = f"""SELECT COUNT(bm.biblionumber)
    FROM koha{year}.biblio_metadata bm
    JOIN koha{year}.biblio b ON b.biblionumber = bm.biblionumber 
    WHERE YEAR(b.datecreated) = '{year}'
    AND ExtractValue( bm.metadata, '//datafield[@tag="801"]/subfield[@code="b"]' ) != '595126101'"""
r = pd.read_sql(query, db_conn)
r

,COUNT(bm.biblionumber)
0,8033


## C3 - Locaux
|Intitulé|valeur|code SLL|
|:--|---|---|
|Surface utile nette totale (en m²)|5300|C301|
|Espaces de services Interns ( en m²)|590|C302|
|Espace de service public (en m²)|3350|C303|
|Espace manifestations et animations (en m²)|250|C304|
|Espace de magains|550|C305|
|Caféterias et surfaces détente|140|C306|
|Votre bâtiment est-il accessible au sens de la loi du 11 février 2005 ?|oui|C309|
|Précisez|Monte-personnes et ascenseurs qui permettent à chacun d'accéder à l'ensemble du bâtiment|C310|


# D - Collections

In [7]:
query = f"""SELECT
                    i.itemnumber,
                    i.barcode,
                    i.dateaccessioned,
                    i.price,
                    i.homebranch,
                    i.holdingbranch,
                    i.location,
                    i.ccode,
                    i.itemcallnumber,
                    i.notforloan,
                    i.damaged,
                    DATE(i.damaged_on),
                    i.withdrawn,
                    DATE(i.withdrawn_on),
                    i.itemlost,
                    DATE(i.itemlost_on),
                    i.onloan,
                    i.datelastborrowed,
                    i.biblionumber,
                    b.title as titre,
                    bi.publicationyear,
                    bi.itemtype,
                    i.timestamp
                FROM koha{year}.items i
                JOIN koha{year}.biblioitems bi ON i.biblionumber = bi.biblionumber
                JOIN koha{year}.biblio b ON i.biblionumber = b.biblionumber
                WHERE i.notforloan IN (-1, -2, 0, 2, -4, -3, 5)
                AND i.homebranch != 'MUS'"""
items = pd.read_sql(query, db_conn)

In [8]:
# On instancie la classe Document
# La classe Document permet d'ajouter les infos dont ont a besoin pour le SLL

exemplaires = Document(df=items, db_conn=db_conn)
exemplaires.get_doc_statdb_data()
exemplaires.get_doc_es_data()

In [9]:
exemplaires.df.columns

Index(['itemnumber', 'barcode', 'dateaccessioned', 'price', 'homebranch',
       'holdingbranch', 'location', 'ccode', 'itemcallnumber', 'notforloan',
       'damaged', 'DATE(i.damaged_on)', 'withdrawn', 'DATE(i.withdrawn_on)',
       'itemlost', 'DATE(i.itemlost_on)', 'onloan', 'datelastborrowed',
       'biblionumber', 'titre', 'publicationyear', 'itemtype', 'timestamp',
       'doc_item_id', 'doc_item_code_barre', 'doc_item_date_creation',
       'doc_item_prix', 'doc_item_site_detenteur_code',
       'doc_item_site_rattachement_code', 'doc_item_localisation_code',
       'doc_item_collection_ccode', 'doc_item_cote', 'doc_statut_code',
       'doc_statut_abime_code', 'doc_statut_desherbe_code',
       'doc_statut_perdu_code', 'doc_usage_emprunt_date',
       'doc_usage_date_dernier_pret', 'doc_biblio_id', 'doc_biblio_titre',
       'doc_biblio_annee_publication', 'doc_biblio_support_code',
       'doc_item_date_modif', 'doc_item_date_creation_annee',
       'doc_item_site_detenteur'

In [10]:
exemplaires.df['sll_acq'] = 'non'
exemplaires.df.loc[exemplaires.df['doc_item_date_creation'].astype('str').str[0:4] == '2023', 'sll_acq' ] = 'oui'

In [11]:
exemplaires_sll = exemplaires.df
exemplaires_sll = exemplaires_sll[['sll_acq', 'sll_public', 'sll_acces', 'sll_collection', 'doc_biblio_id', 'doc_item_id']]

In [12]:
exemplaires_sll['sll_public'].value_counts()

adultes    193890
enfants     71315
Name: sll_public, dtype: int64

In [13]:
exemplaires_sll['sll_acq'].value_counts()

non    251428
oui     13777
Name: sll_acq, dtype: int64

In [14]:
exemplaires_sll['sll_acces'].value_counts()

accès direct      138005
accès indirect    127200
Name: sll_acces, dtype: int64

In [15]:
query = f"""SELECT
                    itemnumber,
                    barcode,
                    dateaccessioned,
                    homebranch,
                    location,
                    ccode,
                    biblionumber,
                    itemtype
                FROM statdb.stat_eliminations
                WHERE annee_mise_pilon = '{year}'"""
elim = pd.read_sql(query, db_conn)

In [16]:
eliminations = Document(df=elim, db_conn=db_conn)
eliminations.get_doc_statdb_data()
eliminations.get_doc_es_data()

In [17]:
eliminations_sll = eliminations.df
eliminations_sll = eliminations_sll[['sll_public', 'sll_acces', 'sll_collection', 'doc_biblio_id', 'doc_item_id']]

## D1 - Livres imprimés

In [18]:
data = exemplaires_sll[exemplaires_sll['sll_collection'] == 'D1 - Livres imprimés']
data_el = eliminations_sll[eliminations_sll['sll_collection'] == 'D1 - Livres imprimés']

### Fonds
#### UM - Adultes et Enfants

In [19]:
data.groupby(['sll_public'])['doc_item_id'].nunique()

sll_public
adultes    126209
enfants     67220
Name: doc_item_id, dtype: int64

#### UM en libre-accès - Adultes et Enfants

In [20]:
data_libre_acces = data[data['sll_acces'] == 'accès direct']
data_libre_acces.groupby(['sll_public'])['doc_item_id'].nunique()

sll_public
adultes    66611
enfants    32628
Name: doc_item_id, dtype: int64

#### TIT en libre accès - Adultes et Enfants

In [21]:
data_libre_acces = data[data['sll_acces'] == 'accès direct']
data_libre_acces.groupby(['sll_public'])['doc_biblio_id'].nunique()

sll_public
adultes    65343
enfants    28945
Name: doc_biblio_id, dtype: int64

### Acquisitions

In [22]:
data[data['sll_acq'] == 'oui'].groupby(['sll_public'])['doc_item_id'].nunique()

sll_public
adultes    5421
enfants    5625
Name: doc_item_id, dtype: int64

In [23]:
data[(data['sll_acq'] == 'oui')
     & (data['sll_acces'] == 'accès direct')].groupby(['sll_public'])['doc_biblio_id'].nunique()

sll_public
adultes    4400
enfants    3787
Name: doc_biblio_id, dtype: int64

#### Eliminations

In [24]:
data_el.groupby(['sll_public'])['doc_item_id'].nunique()

sll_public
adultes    9246
enfants    5183
Name: doc_item_id, dtype: int64

###  TEST LUCAS

In [25]:
data.groupby(['sll_public'])['doc_item_id'].nunique()

sll_public
adultes    126209
enfants     67220
Name: doc_item_id, dtype: int64

In [26]:
import numpy as np
data_fonds = data.pivot_table(index=['sll_public','sll_acces'],
                             values=['doc_item_id','doc_biblio_id'],
                             aggfunc="nunique",
                             #margins=True,
                             #margins_name='Total'
                )
#data_fonds['total'] = data_fonds['doc_biblio_id'] + data_fonds['doc_item_id']

data_fonds_concat = pd.concat([
    y.append(y.sum().rename((x, 'Sous-Total')))
    for x, y in data_fonds.groupby(level=0)]).append(data_fonds.sum().rename(('Enfants + Adultes','Total')))
data_fonds_concat

/tmp/ipykernel_31061/2752056215.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y.append(y.sum().rename((x, 'Sous-Total')))
/tmp/ipykernel_31061/2752056215.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y.append(y.sum().rename((x, 'Sous-Total')))
/tmp/ipykernel_31061/2752056215.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_fonds_concat = pd.concat([


doc_biblio_id  doc_item_id
sll_public        sll_acces                                 
adultes           accès direct            65343        66611
                  accès indirect          56214        59598
                  Sous-Total             121557       126209
enfants           accès direct            28945        32628
                  accès indirect          25145        34592
                  Sous-Total              54090        67220
Enfants + Adultes Total                  175647       193429

## D2 - Publications en séries en cours imprmimées
> Demander à Stéphanie

## D3 - Autres documents

###  Documents cartographiques

In [27]:
data = exemplaires_sll[exemplaires_sll['sll_collection'] == 'D3 – Documents cartographiques']

In [28]:
data['doc_item_id'].nunique()

86

In [29]:
data[data['sll_acq'] == 'oui']['doc_item_id'].nunique()

0

###  Musique imprimée

In [30]:
data = exemplaires_sll[exemplaires_sll['sll_collection'] == 'D3 – Musique imprimée']

In [31]:
data['doc_item_id'].nunique()

3199

In [32]:
data[data['sll_acq'] == 'oui']['doc_item_id'].nunique()

74

In [33]:
data[(data['sll_acces'] == 'accès direct')]['doc_item_id'].nunique()

1126

In [34]:
data[(data['sll_acces'] == 'accès direct')]['doc_biblio_id'].nunique()

1123

In [35]:
data[(data['sll_acces'] == 'accès direct') & (data['sll_acq'] == 'oui')]['doc_biblio_id'].nunique()

73

###  Documents graphiques

In [36]:
data = exemplaires_sll[exemplaires_sll['sll_collection'] == 'D3 - Documents graphiques']

In [37]:
data['doc_item_id'].nunique()

95

In [38]:
data[data['sll_acq'] == 'oui']['doc_item_id'].nunique()

1

###  Autres documents

In [39]:
data = exemplaires_sll[exemplaires_sll['sll_collection'] == 'D3 - Autres documents']

In [40]:
data['doc_item_id'].nunique()

129

In [41]:
data[data['sll_acq'] == 'oui']['doc_item_id'].nunique()

44

### Eliminations

In [42]:
data_el = eliminations_sll[eliminations_sll['sll_collection'].str[:2] == 'D3']
len(data_el)

29

## D4 - Documents audiovisuels sur support

In [43]:
exemplaires_sll['sll_collection'].value_counts()

D1 - Livres imprimés                                                                  193429
D4 - Documents audiovisuels fonds adultes / Documents sonores : musique                44586
D4 - Documents audiovisuels fonds adultes / documents vidéo adultes                    12433
D1 - Publications en série imprimées                                                    7134
D3 – Musique imprimée                                                                   3199
D4 - Documents audiovisuels fonds adultes / Documents sonores : livres enregistrés      1641
D4 - Documents audiovisuels fonds enfants / documents vidéo enfants                     1359
D4 - Documents audiovisuels fonds enfants / Documents sonores : musique                  693
D4 - Total documents multimédia sur support                                              217
D4 - Documents audiovisuels fonds enfants / Documents sonores : livres enregistrés       204
D3 - Autres documents                                                 

In [44]:
d4_coll = exemplaires_sll[exemplaires_sll['sll_collection'].str[:2] == 'D4']['sll_collection'].unique().tolist()

In [45]:
for c in d4_coll:
    print(c)
    print('-------------')
    data = exemplaires_sll[exemplaires_sll['sll_collection'] == c]
    data_el = eliminations_sll[eliminations_sll['sll_collection'] == c]
    print(f"fonds : {len(data)}")
    print(f"acquisitions : {len(data[data['sll_acq'] == 'oui'])}")
    print(f"eliminations : {len(data_el)}")
    
    print('-------------') 

D4 - Documents audiovisuels fonds adultes / Documents sonores : musique
-------------
fonds : 44586
acquisitions : 602
eliminations : 1237
-------------
D4 - Documents audiovisuels fonds enfants / Documents sonores : musique
-------------
fonds : 693
acquisitions : 22
eliminations : 7
-------------
D4 - Documents audiovisuels fonds adultes / Documents sonores : livres enregistrés
-------------
fonds : 1641
acquisitions : 43
eliminations : 53
-------------
D4 - Documents audiovisuels fonds adultes / documents vidéo adultes
-------------
fonds : 12433
acquisitions : 374
eliminations : 191
-------------
D4 - Total documents multimédia sur support
-------------
fonds : 217
acquisitions : 1
eliminations : 16
-------------
D4 - Documents audiovisuels fonds enfants / Documents sonores : livres enregistrés
-------------
fonds : 204
acquisitions : 1
eliminations : 4
-------------
D4 - Documents audiovisuels fonds enfants / documents vidéo enfants
-------------
fonds : 1359
acquisitions : 95
eli

In [46]:
data = exemplaires_sll[exemplaires_sll['sll_collection'] == 'D1 - Livres imprimés']
data_el = eliminations_sll[eliminations_sll['sll_collection'] == 'D1 - Livres imprimés']

## D6 - Ressources numériques acquises
> Depuis juin 2023 les ressources ne sont plus acquises directement par la médiathèque de Roubaix, mais par la MEL.

### Bouquets de contenus
Pour chaque item, merci de sélectionner le bouquet principal si vous en avez plusieurs
 
|Intitulé|oui/non|Nom de l'offre|Code SLL
|:---|:---:|---|---|
    |Livre|non|   |D522
|Livres enregistrés|non|   |D524
|Dictionnaires, encyclopédies|non|   |D526|
|Musique|non|   |D528
|Vidéo|non|   |D530|
|Presse|non|   |D532|
|Apprentissage en ligne (adultes)|non|   |D534|
|Ludo-éducatif (jeunesse)|non|   |D536|
|Autres bouquets acquis|non|   |D538

### Ressources à l'unité

|Intitulé|oui/non|Nombre d'acquisitions|Nom du fournisseur|Code SLL|
|:---|---|---|---|---|
|Livres|oui|45|Autre|D539-540-541|
|Presse|non|   |   |   |
Applications|non|   |   |   |
Autres ressources|non|   |   |    |

## D7 - Collections adaptées

|intitulé|oui/non|Code SLL|
|:---|---|---|
|La bibliothèque propose-t-elle des collections adaptées ?|oui|D701
|Livres en gros caractères|oui|D708
|Livres lus du commerce|oui|D709
|Braille|non|D702
|Ouvrages en Daisy|non|
|Revues en synthèse vocale|non|D703
|Revues en synthèse vocale|non|D704
|Livres-DVD en langue des signes française (LSF)|non|D705
|Livres tactiles ou en relief|non|D706
|Autres collections adaptées|oui|D707

# E - Usages et usagers de la bibliothèque

## E1 - Usagers

In [47]:
query = f"""SELECT
    date_extraction,
    age as adh_age,
    geo_ville as adh_geo_ville,
    geo_roubaix_iris as adh_geo_rbx_iris_code,
    sexe as adh_sexe,
    inscription_code_carte as adh_inscription_carte_code,
    inscription_code_site as adh_inscription_site_code,
    inscription_attribut as adh_inscription_attribut_action_code,
    inscription_fidelite as adh_inscription_nb_annees_adhesion,
    nb_venues_prets_mediatheque,
    nb_venues_prets_bus
FROM statdb.stat_adherents WHERE date_extraction = '{date_extraction_usagers}'"""
inscrits = pd.read_sql(query, db_conn)

In [48]:
adh = Adherent(df=inscrits, db_conn=db_conn)
adh.get_adherent_statdb_data()
adh.get_adherent_es_data()

In [49]:
adh.df['nb'] = 1
adh.df['adh_age_sll'] = 'Non pertinent'
adh.df.loc[adh.df['adh_age_lib3'] == '0 - 14 ans', 'adh_age_sll'] = '0 - 14 ans'
adh.df.loc[adh.df['adh_age_lib3'] == '15 - 24 ans', 'adh_age_sll'] = '15 - 64 ans'
adh.df.loc[adh.df['adh_age_lib3'] == '25 - 64 ans', 'adh_age_sll'] = '15 - 64 ans'
adh.df.loc[adh.df['adh_age_lib3'] == '65 ans et plus', 'adh_age_sll'] = '65 ans et plus'
adh.df.loc[adh.df['adh_age_lib3'] == 'Inconnu', 'adh_age_sll'] = 'Inconnu'

### Inscrits actifs

In [50]:
data = adh.df

In [51]:
data.groupby(['adh_age_sll', 'adh_sexe'])['nb'].sum()

adh_age_sll     adh_sexe
0 - 14 ans                    54
                F           2772
                M           2516
                O              1
15 - 64 ans                  127
                F           5328
                M           3433
                O              6
65 ans et plus                 6
                F            515
                M            417
Non pertinent   N            302
Name: nb, dtype: int64

In [52]:
data.groupby(['adh_age_sll'])['nb'].sum()

adh_age_sll
0 - 14 ans        5343
15 - 64 ans       8894
65 ans et plus     938
Non pertinent      302
Name: nb, dtype: int64

In [53]:
data.groupby(['adh_age_lib2', 'adh_sexe'])['nb'].sum()

adh_age_lib2    adh_sexe
15 ans et plus               133
                F           5843
                M           3850
                O              6
<= 14 ans                     54
                F           2772
                M           2516
                O              1
Non pertinent   N            302
Name: nb, dtype: int64

In [54]:
data[(data['adh_geo_ville'] == 'ROUBAIX') & (~adh.df['adh_age_sll'].isin(['Non pertinent', 'Inconnu']))]['nb'].sum()

10705

### dont nouveaux inscrits

In [55]:
data = adh.df[adh.df['adh_inscription_nb_annees_adhesion'] == 0]
len(data)

6451

In [56]:
data.groupby(['adh_age_sll', 'adh_sexe'])['nb'].sum()

adh_age_sll     adh_sexe
0 - 14 ans                    54
                F           1229
                M           1148
                O              1
15 - 64 ans                  127
                F           2205
                M           1408
                O              5
65 ans et plus                 6
                F            110
                M             94
Non pertinent   N             64
Name: nb, dtype: int64

In [57]:
data.groupby(['adh_age_sll'])['nb'].sum()

adh_age_sll
0 - 14 ans        2432
15 - 64 ans       3745
65 ans et plus     210
Non pertinent       64
Name: nb, dtype: int64

In [58]:
data.groupby(['adh_age_lib2', 'adh_sexe'])['nb'].sum()

adh_age_lib2    adh_sexe
15 ans et plus               133
                F           2315
                M           1502
                O              5
<= 14 ans                     54
                F           1229
                M           1148
                O              1
Non pertinent   N             64
Name: nb, dtype: int64

In [59]:
data.groupby(['adh_age_lib2'])['nb'].sum()

adh_age_lib2
15 ans et plus    3955
<= 14 ans         2432
Non pertinent       64
Name: nb, dtype: int64

In [60]:
data[(data['adh_geo_ville'] == 'ROUBAIX') & (~adh.df['adh_age_sll'].isin(['Non pertinent', 'Inconnu']))]['nb'].sum()

/tmp/ipykernel_31061/3053756961.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data[(data['adh_geo_ville'] == 'ROUBAIX') & (~adh.df['adh_age_sll'].isin(['Non pertinent', 'Inconnu']))]['nb'].sum()


4493

In [61]:
data['adh_inscription_carte_personnalite'].value_counts()

Personne        6387
Collectivité      64
Name: adh_inscription_carte_personnalite, dtype: int64

### Emprunteurs actifs

In [62]:
adh.df.columns

Index(['date_extraction', 'adh_age', 'adh_geo_ville', 'adh_geo_rbx_iris_code',
       'adh_sexe', 'adh_inscription_carte_code', 'adh_inscription_site_code',
       'adh_inscription_attribut_action_code',
       'adh_inscription_nb_annees_adhesion', 'nb_venues_prets_mediatheque',
       'nb_venues_prets_bus', 'adh_inscription_carte_personnalite_code',
       'adh_age_code', 'adh_age_lib1', 'adh_age_lib2', 'adh_age_lib3',
       'adh_geo_gentilite', 'adh_geo_rbx_iris', 'adh_geo_rbx_quartier',
       'geo_rbx_secteur', 'inscription_carte', 'adh_inscription_carte_type',
       'adh_inscription_carte_gratuite', 'adh_inscription_carte_prix',
       'adh_inscription_carte_personnalite', 'adh_inscription_site',
       'adh_inscription_nb_annees_adhesion_tra',
       'adh_inscription_attribut_action', 'nb', 'adh_age_sll'],
      dtype='object')

In [63]:
adh.df['emprunteur'] = 'non'
adh.df.loc[adh.df['nb_venues_prets_bus'] > 0, 'emprunteur'] = 'oui'
adh.df.loc[adh.df['nb_venues_prets_mediatheque'] > 0, 'emprunteur'] = 'oui'

data = adh.df[adh.df['emprunteur'] == 'oui']
len(data)

9993

In [64]:
data.groupby(['adh_age_sll', 'adh_sexe'])['nb'].sum()

adh_age_sll     adh_sexe
0 - 14 ans                    30
                F           2128
                M           1838
15 - 64 ans                   42
                F           3260
                M           1685
                O              3
65 ans et plus                 3
                F            434
                M            285
Non pertinent   N            285
Name: nb, dtype: int64

In [65]:
data.groupby(['adh_age_sll'])['nb'].sum()

adh_age_sll
0 - 14 ans        3996
15 - 64 ans       4990
65 ans et plus     722
Non pertinent      285
Name: nb, dtype: int64

In [66]:
data.groupby(['adh_age_lib2', 'adh_sexe'])['nb'].sum()

adh_age_lib2    adh_sexe
15 ans et plus                45
                F           3694
                M           1970
                O              3
<= 14 ans                     30
                F           2128
                M           1838
Non pertinent   N            285
Name: nb, dtype: int64

In [67]:
data.groupby(['adh_age_lib2'])['nb'].sum()

adh_age_lib2
15 ans et plus    5712
<= 14 ans         3996
Non pertinent      285
Name: nb, dtype: int64

In [68]:
data[(data['adh_geo_ville'] == 'ROUBAIX') & (~adh.df['adh_age_sll'].isin(['Non pertinent', 'Inconnu']))]['nb'].sum()

/tmp/ipykernel_31061/3053756961.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data[(data['adh_geo_ville'] == 'ROUBAIX') & (~adh.df['adh_age_sll'].isin(['Non pertinent', 'Inconnu']))]['nb'].sum()


6677

In [69]:
data['adh_inscription_carte_personnalite'].value_counts()

Personne        9708
Collectivité     285
Name: adh_inscription_carte_personnalite, dtype: int64

### Fréquentation : entrées dans l'établissement

In [70]:
query = f"""SELECT
    SUM(entrees)
FROM statdb.stat_entrees WHERE YEAR(datetime) = '{year}'"""
entrees = pd.read_sql(query, db_conn)
entrees

,SUM(entrees)
0,258427.0


## E2 - Prêts

In [88]:
query = f"""SELECT
    issuedate as pret_date,
    date_due as pret_date_retour_prevue,
    returndate as pret_date_retour_effectif,
    renewals as pret_nb_renouvellement,
    branch as pret_site_pret_code,
    age as adh_age,
    sexe as adh_sexe_code,
    ville as adh_geo_ville,
    iris as adh_geo_rbx_iris_code,
    branchcode as adh_inscription_site_code,
    categorycode as adh_inscription_carte_code,
    fidelite as adh_inscription_nb_annees_adhesion,
    itemnumber as doc_item_id,
    homebranch as doc_item_site_detenteur_code,
    location as doc_item_localisation_code,
    ccode as doc_item_collection_ccode,
    itemcallnumber as doc_item_cote,
    itemtype as doc_biblio_support_code,
    publicationyear as doc_biblio_annee_publication,
    biblionumber as doc_biblio_id,
    dateaccessioned as doc_item_date_creation
FROM statdb.stat_issues
WHERE YEAR(issuedate) = '{year}'"""
prts = pd.read_sql(query, db_conn)
len(prts)

364269

In [89]:
c2l = Code2Libelle(db_conn)
c2l.get_val()
c2l = c2l.dict_codes_lib

prets = Pret(df=prts, db_conn=db_conn, c2l=c2l)
prets.get_pret_statdb_data()
prets.get_pret_es_data()

In [90]:
prets_sll = prets.df[['sll_public', 'sll_acces', 'sll_prets_coll', 'sll_collection', 'sll_prets']]
prets_sll['nb'] = 1

/tmp/ipykernel_31061/3051755607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prets_sll['nb'] = 1


In [131]:
# Nb de prêts - hors Collectivités
data = prets_sll.pivot_table(index="sll_prets",
                             columns="sll_public",
                             values="nb",
                             aggfunc=sum,
                             margins=True,
                             margins_name='Total')
data

sll_public,adultes,enfants,Total
sll_prets,,,
Autres documents,1047,85,1132
E2 - Documents vidéo,18106,4073,22179
E2 - Livres,105793,201203,306996
E2 – Documents sonores : livres,1472,314,1786
E2 – Documents sonores : musique,13305,491,13796
E2 – Publications en série imprimées,12900,5480,18380
Total,152623,211646,364269


In [132]:
# Nb de prêts - Collectivités
data_coll = prets_sll[prets_sll['sll_prets_coll']=="Prêt aux collectivités"].pivot_table(index="sll_prets",
                                                                                         #columns="sll_public",
                                                                                         values="nb",
                                                                                         aggfunc=sum,
                                                                                         margins=True,
                                                                                         margins_name='Total').rename(columns={"nb":"Dont Collectivités"})
data_coll

,Dont Collectivités
sll_prets,
Autres documents,84
E2 - Documents vidéo,81
E2 - Livres,29290
E2 – Documents sonores : livres,28
E2 – Documents sonores : musique,24
E2 – Publications en série imprimées,52
Total,29559


In [136]:
data_synthese = data.merge(data_coll,
                           left_on='sll_prets',
                           right_on='sll_prets',
                           how='left'
                          )
data_synthese = data_synthese[["adultes",'enfants','Total','Dont Collectivités']]

In [138]:
lst = ["E2 - Livres",
       "E2 – Publications en série imprimées",
       "E2 – Documents sonores : musique",
       "E2 – Documents sonores : livres",
       "E2 - Documents vidéo",
       "Autres documents",
       "Total"
      ]
data_synthese.loc[lst]

,adultes,enfants,Total,Dont Collectivités
sll_prets,,,,
E2 - Livres,105793,201203,306996,29290
E2 – Publications en série imprimées,12900,5480,18380,52
E2 – Documents sonores : musique,13305,491,13796,24
E2 – Documents sonores : livres,1472,314,1786,28
E2 - Documents vidéo,18106,4073,22179,81
Autres documents,1047,85,1132,84
Total,152623,211646,364269,29559


> A la dernière ligne **Tota** il faut ajouter pour valeur le **Nb de prêts de livres numériques**. Ce chiffre est à récupérer auprès de Mathilde L.

## E3 - Autres types d'usages

### Réservations

In [85]:
query = f"""SELECT
    COUNT(*)
FROM statdb.stat_reserves WHERE YEAR(reservedate) = '{year}'"""
r = pd.read_sql(query, db_conn)
r

,COUNT(*)
0,43341


## E5 - Sessions internet (connexions + wifi)

In [86]:
query = f"""SELECT
    COUNT(*)
FROM statdb.stat_webkiosk WHERE YEAR(heure_deb) = '{year}'"""
r = pd.read_sql(query, db_conn)
r

,COUNT(*)
0,55961


In [87]:
query = f"""SELECT
    COUNT(*)
FROM statdb.stat_wifi WHERE YEAR(start_wifi) = '{year}'"""
r = pd.read_sql(query, db_conn)
r

,COUNT(*)
0,13988


### E502 - Nombre de visites du site dans la bibliothèque (visite interne)

In [88]:
query = f"""SELECT SUM(visites) AS 'Total visites interne'
FROM statdb.stat_web2 sw 
WHERE YEAR(`date`) = {year}
AND origine = 'interne'"""

pd.read_sql(query,db_conn)

,Total visites interne
0,13920.0


### E503 - Nombre de visite du site internet hors bibliothèque (visite externe)

In [89]:
query = f"""SELECT SUM(visites) AS 'Total visites externe'
FROM statdb.stat_web2 sw 
WHERE YEAR(`date`) = {year}
AND origine = 'externe'"""

pd.read_sql(query,db_conn)

,Total visites externe
0,175338.0


# H - Action culturelle (nouvelle version)

In [113]:
import pandas as pd

In [114]:
ac = pd.read_excel("/home/kibini/AC_synthese_2023-2024.xlsx")

/home/kibini/miniconda3/envs/kibini/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [115]:
ac = ac[ac['Date'].dt.year==2024]

In [116]:
ac.groupby("Nouveau Type d'action").size()

Nouveau Type d'action
Atelier                 56
Atelier d'écriture       6
Atelier informatique     9
Autre                    2
Club lecture             7
Concert                 11
Conférence              25
Diffusion                6
Exposition               3
Jeux                    10
Jeux-vidéo              27
Lecture                  9
Projection               4
Rencontre               11
Spectacle               16
Séance de contes        20
Vente                    2
dtype: int64

In [117]:
# SLL Types d'action
ac.loc[ac["Nouveau Type d'action"]=="Exposition",["sll_type_action"]] = "H4 - Exposition"

ac.loc[ac["Nouveau Type d'action"].isin(['Conférence','Rencontre','Lecture']),['sll_type_action']] = "H4 - Conférences, rencontres, lectures"

ac.loc[ac["Nouveau Type d'action"].isin(['Concert','Projection']),['sll_type_action']] = "H4 - Concerts, projections"

ac.loc[ac["Nouveau Type d'action"]=="Séance de contes",["sll_type_action"]] = "H4 - Séances de contes"

ac.loc[ac["Nouveau Type d'action"].isin(["Club lecture","Atelier d'écriture"]),['sll_type_action']] = "H4 - Clubs de lecteurs, ateliers d'écriture"

#ac.loc[ac["Nouveau Type d'action"].isin(["Journée festive","Salon du livre","Festival"]),['sll-type_action']] = "H4 - Fêtes, salons du livre, festivals" 
#ac.loc[ac["Evénement"].notna(),["sll_type_action"]] = "H4 - Fêtes, salons du livre, festivals"


ac.loc[~ac["sll_type_action"].isin(["H4 - Exposition",
                                    "H4 - Conférences, rencontres, lectures",
                                    "H4 - Concerts, projections",
                                    "H4 - Séances de contes",
                                    "H4 - Clubs de lecteurs, ateliers d'écriture",
                                    #"H4 - Fêtes, salons du livre, festivals"
                                   ]),
       ["sll_type_action"]] = "H4 - Autres"

> La source de donnée utilisée pour réaliser ces stats est amené à évoluer. Il faudra bien penser à finaliser ce script une fois que les noms de colonnes du fichier Excel Action Culturel seront définitivement fixés

In [118]:
# SLL Types de publics

ac.loc[ac["Public"].isin(["Adolescents","Adultes","Familles","Tout public"]),["sll_type_public"]] = "a/ Tout public"
ac.loc[ac['Public'].isin(["Enfants","Petite enfance"]),["sll_type_public"]] = "b/ Enfants"

In [119]:
ac['Nb actions'] = 1

In [120]:
# import du fichier de suivi des actions AECS
sheets2keep = ['Bruno',
               'Céline',
               'Esther',
               'Gwen',
               'Hélène',
               'Inès',
               'Laetitia C',
               'Laetitia D',
               'Mathilde M',
               'Pascale',
               'Zina',
              ]

aecs = pd.read_excel("/home/kibini/formulaire_AECS.xlsx",sheets2keep)
aecs

/home/kibini/miniconda3/envs/kibini/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


{'Bruno':    Date début action      Date fin action          Champ d'action  \
 0         2024-10-04  2023-10-18 00:00:00   dont Livre et lecture   
 1         2024-10-13  2023-05-24 00:00:00               Education   
 2         2024-10-13  2023-05-24 00:00:00               Education   
 3         2024-10-20  2023-05-23 00:00:00               Education   
 4         2024-10-20  2023-05-23 00:00:00               Education   
 5         2024-11-09  2023-11-09 00:00:00               Education   
 6         2024-11-17          17/11/20223   dont Livre et lecture   
 7         2024-12-07  2023-12-15 00:00:00               Education   
 8         2024-12-14  2024-02-25 00:00:00   dont Livre et lecture   
 9         2024-11-08  2024-12-13 00:00:00   dont Livre et lecture   
 10        2024-01-10  2024-02-21 00:00:00   dont Livre et lecture   
 11        2023-12-11  2024-02-02 00:00:00               Education   
 12        2024-02-01  2024-03-29 00:00:00   dont Livre et lecture   
 13        

In [121]:
aecs = pd.concat(aecs)

In [122]:
effectifs_scolaires = pd.read_excel("/home/kibini/formulaire_AECS.xlsx",sheet_name="Effectifs")
effectifs_scolaires

,Nom,Total Maternelle,Total Elementaire,Total Ecole
0,Albert Camus,215.0,324.0,539
1,Albert Samain,150.0,0.0,150
2,Alphonse Daudet,24.0,0.0,24
3,Anatole France,112.0,240.0,352
4,Blaise Pascal,57.0,104.0,161
5,Boileau / Pasteur,156.0,295.0,451
6,Buffon,127.0,219.0,346
7,Charles De Foucauld,NaN,NaN,0
8,Charles Perrault,32.0,NaN,32
9,Condorcet,NaN,137.0,137


In [123]:
effectifs_scolaires.loc[effectifs_scolaires['Nom']=='Saint Louis']["Total Elementaire"].values[0]

148.0

In [124]:
aecs.columns

Index(['Date début action', 'Date fin action', 'Champ d'action',
       'Type d'action', 'Nom action ou projet', 'Lieu',
       'Espace (si hors les murs et pas dans la structure partenaire',
       'Type de public', 'Niveau si école', 'Type d'équipement', 'Structure',
       'Nom d'équipement', 'Nom du porteur de l'action',
       'Nb de classes touchées si public scolaire', 'Nb enfants touchés',
       'Nb adultes touchés', 'Nb d'accueil ou séance',
       'Total personnes touchées', 'Notes', 'Code de malle', 'Titre de malle',
       'Nom emprunteur', 'Nombre de docs', 'Evenement',
       'Espace (si dans les murs)', 'Partenariat', 'Jauge'],
      dtype='object')

### Calcul des actions collectivités

In [125]:
# Remise à plat du df
aecs = aecs.reset_index()

In [126]:
aecs.drop(labels='level_1',axis='columns',inplace=True)

In [127]:
aecs.rename(columns={'level_0':'nom_feuille'},inplace=True)

In [128]:
# Filtre Malles Collectivités
aecs_coll = aecs[aecs["nom_feuille"]=='Laetitia D']

# Sélection des structures écoles

# Liste des noms d'école
school_names = aecs_coll[aecs_coll["Structure"]=='Ecole']["Nom d'équipement"].unique()

for name in school_names:
    # On définit un aecs qui filtre à partir de l'itération school_name
    aecs_coll_school = aecs_coll[aecs_coll["Nom d'équipement"]==name]
    # On calcule un total par types d'actions malles
    Total_actions_BCD = len(aecs_coll_school[aecs_coll_school['Nom action ou projet']=='BCD'])
    Total_actions_MARM = len(aecs_coll_school[aecs_coll_school['Nom action ou projet']=='MARMOTHEQUE'])
    Total_emprunteurs_distincts = aecs_coll_school['Nom emprunteur'].nunique()
    
    # On insère dans le dataframe de base les colonnes du dataframe par école
    aecs_coll.loc[aecs_coll["Nom d'équipement"]==name,['Total_actions_BCD']] = Total_actions_BCD
    aecs_coll.loc[aecs_coll["Nom d'équipement"]==name,['Total_actions_MARM']] = Total_actions_MARM
    aecs_coll.loc[aecs_coll["Nom d'équipement"]==name,['Total_emprunteurs_distincts']] = Total_emprunteurs_distincts

aecs_coll

,nom_feuille,Date début action,Date fin action,Champ d'action,Type d'action,Nom action ou projet,Lieu,Espace (si hors les murs et pas dans la structure partenaire,Type de public,Niveau si école,...,Titre de malle,Nom emprunteur,Nombre de docs,Evenement,Espace (si dans les murs),Partenariat,Jauge,Total_actions_BCD,Total_actions_MARM,Total_emprunteurs_distincts
249,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),CP,...,Emotions et sentiments,HUCHETTE Juliette,28,NaN,NaN,NaN,NaN,0.0,0.0,15.0
250,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),MS,...,Romans policiers,DRONNEAU Anne-Chrystel,14,NaN,NaN,NaN,NaN,0.0,0.0,15.0
251,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),GS,...,Livres jeux,DEREUMAUX Camille,19,NaN,NaN,NaN,NaN,0.0,0.0,15.0
252,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),CP,...,Afrique,DUFOURMONT Angélique,28,NaN,NaN,NaN,NaN,0.0,0.0,15.0
253,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),CP,...,Abécédaire,ZIARNOWSKI Delphine,23,NaN,NaN,NaN,NaN,0.0,0.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933,Laetitia D,NaT,NaT,NaN,NaN,Malles,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1934,Laetitia D,NaT,NaT,NaN,NaN,Malles,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1935,Laetitia D,NaT,NaT,NaN,NaN,Malles,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936,Laetitia D,NaT,NaT,NaN,NaN,Malles,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
aecs_coll_merged = aecs_coll.merge(effectifs_scolaires,left_on="Nom d'équipement",right_on="Nom",how="left")

In [130]:
# Calcule des publics touchés

effectif_moyen_classe = 25


# Si au moins 1 action MARM : Compter l'effectif Total Maternlle
aecs_coll_merged.loc[aecs_coll_merged['Total_actions_MARM']>0,['Nb enfants touchés']] = aecs_coll_merged['Total Maternelle']

# Si au moins 1 action BCD : Compter l'effectif total Ecole
aecs_coll_merged.loc[aecs_coll_merged['Total_actions_BCD']>0,['Nb enfants touchés']] = aecs_coll_merged['Total Ecole']

# Aucune action MARM ou BCD : Estimation = Nb emprunteurs distincts x Moyenne d'une classe
aecs_coll_merged.loc[(aecs_coll_merged['Total_actions_MARM']==0)&
            (aecs_coll_merged['Total_actions_BCD']==0),['Nb enfants touchés']] = aecs_coll_merged['Total_emprunteurs_distincts'] * effectif_moyen_classe

aecs_coll_merged

,nom_feuille,Date début action,Date fin action,Champ d'action,Type d'action,Nom action ou projet,Lieu,Espace (si hors les murs et pas dans la structure partenaire,Type de public,Niveau si école,...,Espace (si dans les murs),Partenariat,Jauge,Total_actions_BCD,Total_actions_MARM,Total_emprunteurs_distincts,Nom,Total Maternelle,Total Elementaire,Total Ecole
0,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),CP,...,NaN,NaN,NaN,0.0,0.0,15.0,Jules Guesde,123.0,200.0,323.0
1,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),MS,...,NaN,NaN,NaN,0.0,0.0,15.0,Jules Guesde,123.0,200.0,323.0
2,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),GS,...,NaN,NaN,NaN,0.0,0.0,15.0,Jules Guesde,123.0,200.0,323.0
3,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),CP,...,NaN,NaN,NaN,0.0,0.0,15.0,Jules Guesde,123.0,200.0,323.0
4,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),CP,...,NaN,NaN,NaN,0.0,0.0,15.0,Jules Guesde,123.0,200.0,323.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,Laetitia D,NaT,NaT,NaN,NaN,Malles,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1685,Laetitia D,NaT,NaT,NaN,NaN,Malles,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1686,Laetitia D,NaT,NaT,NaN,NaN,Malles,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1687,Laetitia D,NaT,NaT,NaN,NaN,Malles,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
# Suppression des noms doublons pour les noms d'école
aecs_coll_merged.drop_duplicates(subset="Nom d'équipement",
                                 inplace=True
                                )

In [132]:
# tableau spécial collectivités
aecs_coll_merged

,nom_feuille,Date début action,Date fin action,Champ d'action,Type d'action,Nom action ou projet,Lieu,Espace (si hors les murs et pas dans la structure partenaire,Type de public,Niveau si école,...,Espace (si dans les murs),Partenariat,Jauge,Total_actions_BCD,Total_actions_MARM,Total_emprunteurs_distincts,Nom,Total Maternelle,Total Elementaire,Total Ecole
0,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),CP,...,NaN,NaN,NaN,0.0,0.0,15.0,Jules Guesde,123.0,200.0,323.0
6,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),CE2,...,NaN,NaN,NaN,0.0,0.0,8.0,Léo Lagrange,72.0,178.0,250.0
8,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),PS,...,NaN,NaN,NaN,0.0,0.0,6.0,Pierre Brossolette,80.0,148.0,228.0
10,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),CP,...,NaN,NaN,NaN,2.0,0.0,3.0,Léon Jouhaux,NaN,113.0,113.0
12,Laetitia D,2023-09-05 00:00:00,2023-10-17 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),CE2,...,NaN,NaN,NaN,2.0,4.0,16.0,Lavoisier,NaN,266.0,266.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,Laetitia D,2024-10-16 00:00:00,2025-04-23 00:00:00,NaN,NaN,MARMOTHEQUE,Hors les murs,NaN,Petite enfance (0-3 ans),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1138,Laetitia D,2024-10-16 00:00:00,2025-04-23 00:00:00,NaN,NaN,MARMOTHEQUE,Hors les murs,NaN,Petite enfance (0-3 ans),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1139,Laetitia D,2024-10-17 00:00:00,2025-06-30 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1243,Laetitia D,2024-11-07 00:00:00,2024-12-19 00:00:00,NaN,NaN,Malles,Hors les murs,NaN,Enfant Scolaires (3-12 ans),PS,...,NaN,NaN,NaN,0.0,0.0,1.0,Saint Louis,76.0,148.0,224.0


In [133]:
# Afficher le nombre de personnes touchées par école
aecs_coll_merged[aecs_coll_merged["Structure"]=="Ecole"][["Nom d'équipement","Nb enfants touchés"]]

,Nom d'équipement,Nb enfants touchés
0,Jules Guesde,375.0
6,Léo Lagrange,200.0
8,Pierre Brossolette,150.0
10,Léon Jouhaux,113.0
12,Lavoisier,266.0
21,Jean Macé,231.0
30,Marie Auxiliatrice,100.0
34,Michelet,300.0
44,Pierre de Ronsard niv.2,154.0
45,Montesquieu,116.0


In [134]:
nb_actions_prets_malles_2024 = aecs_coll_merged['Nb enfants touchés'].sum()

In [135]:
nb_actions_prets_malles_2024

8413.0

### Fin du calcul

In [136]:
# Rajout d'une colonne pour compter le nombre de valeurs
aecs['nb'] = 1

In [137]:
# Si le champ Nb de séance n'est pas rempli, on considère qu'il s'agit d'une occurence unique
aecs.loc[aecs["Nb d'accueil ou séance"].isna(),["Nb d'accueil ou séance"]] = 1

In [138]:
aecs["Total personnes touchées"] = aecs["Nb enfants touchés"] + aecs["Nb adultes touchés"]

## H1 - Etablissements scolaires et universitaires

## H2 - Equipements culturels 

## H3 - Structures associatives

In [139]:
(ac.pivot_table(index='sll_type_action',
               columns='sll_type_public',
               values=['Nb actions','Nombre de personnes venues'],
              aggfunc=sum,
               margins=True,
               fill_value=0,
               margins_name='Total')
 .astype(int))

Nb actions                   \
sll_type_public                             a/ Tout public b/ Enfants Total   
sll_type_action                                                               
H4 - Autres                                            113         15   122   
H4 - Clubs de lecteurs, ateliers d'écriture             12          1    12   
H4 - Concerts, projections                              10          5    14   
H4 - Conférences, rencontres, lectures                  43          2    44   
H4 - Exposition                                          3          0     3   
H4 - Séances de contes                                   4         16    19   
Total                                                  176         38   214   

                                            Nombre de personnes venues  \
sll_type_public                                         a/ Tout public   
sll_type_action                                                          
H4 - Autres                                                       4831   
H4 - Clubs de lecteurs, ateliers d'écriture                        130   
H4 - Concerts, projections                                         402   
H4 - Conférences, rencontres, lectures                             911   
H4 - Exposition                                                     65   
H4 - Séances de contes                                             766   
Total                                                             7105   

                                                              
sll_type_public                             b/ Enfants Total  
sll_type_action                                               
H4 - Autres                                        499  5330  
H4 - Clubs de lecteurs, ateliers d'écriture          8   138  
H4 - Concerts, projections                         353   755  
H4 - Conférences, rencontres, lectures              23   934  
H4 - Exposition                                      0    65  
H4 - Séances de contes                             663  1429  
Total                                             1546  8651

## H5 - Actions hors de l'établissement

In [141]:
aecs.groupby("Lieu")[("Nb d'accueil ou séance","Total personnes touchées")].sum()

/tmp/ipykernel_846/2387532572.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  aecs.groupby("Lieu")[("Nb d'accueil ou séance","Total personnes touchées")].sum()


,Nb d'accueil ou séance,Total personnes touchées
Lieu,,
Dans les murs,449.0,7777.0
Hors les murs,1734.0,585.0
Les deux,70.0,939.0


In [150]:
nb_actions_hors_les_murs = aecs[aecs["Lieu"]=='Hors les murs']['Total personnes touchées'].sum() + nb_actions_prets_malles_2024

In [151]:
nb_actions_hors_les_murs

8998.0

### Action hors les murs

### Portage à domicile
Le portage à domicile est effectué par M. qui transmet à F. les chiffres chaque année.

## H6 - Actions de coopération

|Intitulé|Oui/Non?|Champ|
|:---|---|---|
|Coopération locale|Oui|H601
|- Dont dont actions de coopération intercommunale (avec des bibliothèques n’appartenant pas à votre collectivité)|Oui|H610|
|Catalogue commun|Oui|H612|
|Portail documentaire commun|Non|H613|
|Politique documentaire commune|Non|H613
|Navette documentaire|Non|H614
|Carte unique de lecteur|Non|H615
|Action culturelle commune|Oui|H617
|Autre action de coopération intercommunal|Non|H616
|Coopération nationale|Oui|H604
|Nombre d'actions|4|H605
|Coopération internationale|Non|H607

## H7 - Actions et services à destination de publics à besoins spécifiques 

---
## H4 - Action culturelle (ANCIENNE VERSION)

In [90]:
query = f"SELECT * FROM statdb.stat_action_culturelle WHERE YEAR(`date`)={year}"

In [91]:
ac_2023 = pd.read_sql(query,db_conn)

## H2 - Etablissements culturels

> Pour l'année 2023, je vais sortir une liste à compléter par Mathilde M. pour lister les partenaires par types d'établissement culturel

> Il faudra ensuite importé le tableau complété et faire les requêtes nécessaires

In [92]:
ac_partenariats_23 = ac_2023.groupby(["type","partenariat"])["participants"].sum().to_frame()

In [93]:
ac_partenariats_23.to_excel("../data_lucas/liste_partenariats_action_culturelle_2023_V2.xlsx")

In [94]:
# Création d'une liste des partenaires et du nombre de participant pour chacun
ac_2023.groupby('partenariat')['participants'].sum("total_participants").to_frame().to_excel('../data_lucas/liste_partenariats_action_culturelle_2023.xlsx')

In [95]:
#Import de la liste complété


## H4 Actions au sein de l'établissement

In [96]:
# Ajouter une colonne Conférences, rencontres, lecture = Conférence + Rencontre/conférence + lecture

In [97]:
# Recréer des catégories d'usagers sur la base des données suivantes : 
# Tous publics = toutes les actions qui ne sont pas Enfants
# Enfants = 0-14 ans
# Tous publics = inverse de Enfants et PetiteEnfance
# Enfants = Enfants + Petite Enfance

In [98]:
ac_2023['public_sll'] = np.NaN

In [99]:
ac_2023.loc[ac_2023['public'].isin(['Enfants','Petite enfance']),['public_sll']] = 'Enfants' 

In [100]:
ac_2023.loc[~ac_2023['public'].isin(['Enfants','Petite enfance']),['public_sll']] = 'Tous publics' 

In [101]:
ac_2023.head(5)

,id,date,action,lieu,type,public,partenariat,participants,jauge,evenement,notes,public_sll
0,1613,2024-01-03,Jeux vidéo,La criée,Jeux,Tout public,None,60.0,NaN,None,None,Tous publics
1,1614,2024-01-06,Jeux vidéo,La criée,Jeux,Tout public,None,55.0,NaN,None,None,Tous publics
2,1615,2024-01-13,39-45 : elles n'ont rien oublié,La criée,Projection/diffusion,Adultes,None,80.0,45.0,None,None,Tous publics
3,1616,2024-01-14,Tricot des Lulus,La criée,Atelier,Tout public,None,22.0,20.0,None,None,Tous publics
4,1617,2024-01-16,Atelier informatique pour débutants,Espace multimédia,Atelier,Adultes,None,10.0,12.0,None,None,Tous publics


In [102]:
ac_2023['nb_actions'] = 1

In [103]:
# Séances de Contes = Lectures/Contes + Racontées (demander ces chiffres à Chantale et/ou Laetitia)
# Formations = Ateliers Action culturelle + Ateliers informatiques (demander ces chiffres à Maïté)

In [104]:
ac_2023_hors_evenement = ac_2023[ac_2023['evenement'].isna()]

In [105]:
ac_2023_hors_evenement

,id,date,action,lieu,type,public,partenariat,participants,jauge,evenement,notes,public_sll,nb_actions
0,1613,2024-01-03,Jeux vidéo,La criée,Jeux,Tout public,None,60.0,NaN,None,None,Tous publics,1
1,1614,2024-01-06,Jeux vidéo,La criée,Jeux,Tout public,None,55.0,NaN,None,None,Tous publics,1
2,1615,2024-01-13,39-45 : elles n'ont rien oublié,La criée,Projection/diffusion,Adultes,None,80.0,45.0,None,None,Tous publics,1
3,1616,2024-01-14,Tricot des Lulus,La criée,Atelier,Tout public,None,22.0,20.0,None,None,Tous publics,1
4,1617,2024-01-16,Atelier informatique pour débutants,Espace multimédia,Atelier,Adultes,None,10.0,12.0,None,None,Tous publics,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,1926,2024-12-20,Radio à La Grand-Plage,Salle VDM,Atelier,Adultes,None,4.0,15.0,None,None,Tous publics,1
214,1927,2024-12-21,Atelier de conversation en français,Salle VDM,Atelier,Adultes,None,7.0,10.0,None,None,Tous publics,1
215,1928,2024-12-21,Petit déjeuner jeux vidéo,La criée,Jeux,Familles,None,12.0,12.0,None,None,Tous publics,1
216,1929,2024-12-21,Les racontées musicales,Salle d'heure du conte,Lecture/contes,Petite enfance,None,48.0,40.0,None,25 enfants et 23 adultes,Enfants,1


> Si on compte dans action dans l'établissement les fêtes etc... Alors pour tous les dataframes sauf le dataframe Fêtes etc... utiliser `ac_2023.loc[ac_2023_hors_evenement]`. Cela permettra de ne pas compter 2 fois le même événement.

### H401 - Expositions

In [106]:
ac_2023.loc[ac_2023['type']=='Exposition',['type_sll']] = 'Exposition'

In [107]:
# Nombre d'exposition dans l'année pour Tous publics
len(ac_2023[(ac_2023['type']=='Exposition')&
            (ac_2023['public_sll']=='Tous publics')]
   )

6

In [108]:
# Nombre d'exposition dans l'année pour enfants
len(ac_2023[(ac_2023['type']=='Exposition')&
            (ac_2023['public_sll']=='Enfants')]
   )

0

### H409 - Conférences, rencontres, lectures

In [109]:
ac_2023.loc[ac_2023['type'].isin(['Conférence','Lecture','Rencontre/conférence']),['type_sll']] = 'Conférences, rencontres, lectures'

In [110]:
# Nombre de conférences, rencontres et lectures TOUT PUBLIC
len(ac_2023[(ac_2023['type_sll']=='Conférences, rencontres, lectures')&
            (ac_2023['public_sll']=='Tous publics')]
   )

30

In [111]:
# Nombre de conférences, rencontres et lectures Enfants
len(ac_2023[(ac_2023['type_sll']=='Conférences, rencontres, lectures')&
        (ac_2023['public_sll']=='Enfants')]
   )

0

In [112]:
# Population touchée
ac_2023.groupby(['type_sll','public_sll'])['participants'].sum()

type_sll                           public_sll  
Conférences, rencontres, lectures  Tous publics    674.0
Exposition                         Tous publics    123.0
Name: participants, dtype: float64

### H413 - Concerts projections

In [113]:
ac_2023.loc[ac_2023['type'].isin(['Concert','Projection','Projection/diffusion']),['type_sll']] = 'Concerts, projections'

In [114]:
len(ac_2023[(ac_2023['type_sll']=='Concerts, projections')&
            (ac_2023['public_sll']=='Tous publics')
           ]
   )

16

In [115]:
len(ac_2023[(ac_2023['type_sll']=='Concerts, projections')&
            (ac_2023['public_sll']=='Enfants')
           ]
   )

8

In [116]:
ac_2023.groupby(['type_sll','public_sll'])['participants'].sum()

type_sll                           public_sll  
Concerts, projections              Enfants         514.0
                                   Tous publics    446.0
Conférences, rencontres, lectures  Tous publics    674.0
Exposition                         Tous publics    123.0
Name: participants, dtype: float64

### H417 - Séances de conte

In [117]:
ac_2023.loc[(ac_2023['type']=='Lecture/contes'),['type_sll']] = 'Séances de conte'

In [118]:
len(ac_2023[(ac_2023['type']=='Séances de conte')&
            (ac_2023['public']=='Tout public')
           ]
   )

0

In [119]:
len(ac_2023[(ac_2023['type']=='Séances de conte')&
            (ac_2023['public']=='Enfants')
           ]
   )

0

In [120]:
ac_2023[ac_2023['type']=='Séances de conte']['participants'].sum()

0.0

### H420 - Clubs de lecteurs ateliers d'écriture

In [121]:
ac_2023.loc[ac_2023['type']=='Club lecture',['type_sll']] = 'Club lecture'

In [122]:
# Nombre total de clubs lectures TOUT PUBLIC
len(ac_2023[(ac_2023['type']=='Club lecture')&
            (ac_2023['public_sll']=='Tous publics')
           ])

7

In [123]:
# Nombre total de clubs lectures ENFANTS
len(ac_2023[(ac_2023['type']=='Club lecture')&
            (ac_2023['public_sll']=='Enfants')
           ])

0

In [124]:
ac_2023[ac_2023['type']=='Club lecture']['participants'].sum()

87.0

### H424 - Fêtes, salons du livre, festivals



In [125]:
ac_2023.loc[ac_2023['evenement'].notna(),['type_sll']] = 'Fêtes, salons du livre, festivals'

In [126]:
ac_2023[ac_2023['type_sll']=='Fêtes, salons du livre, festivals']

,id,date,action,lieu,type,public,partenariat,participants,jauge,evenement,notes,public_sll,nb_actions,type_sll
6,1619,2024-01-20,Bulles de lecture - à partir de 4 ans,Espace RDC,Lecture/contes,Enfants,None,25.0,NaN,Nuits de la lecture,None,Enfants,1,"Fêtes, salons du livre, festivals"
7,1620,2024-01-20,Bulles de lecture 0-3 ans,Salle d'heure du conte,Lecture/contes,Enfants,None,17.0,NaN,Nuits de la lecture,10 enfants et 7 adultes,Enfants,1,"Fêtes, salons du livre, festivals"
8,1621,2024-01-20,Zik et bouquins,La criée,Lecture/contes,Familles,None,39.0,NaN,Nuits de la lecture,None,Tous publics,1,"Fêtes, salons du livre, festivals"
9,1622,2024-01-20,"Between, Ballet du Nord",Espace RDC,Lecture/contes,Tout public,None,26.0,NaN,Nuits de la lecture,None,Tous publics,1,"Fêtes, salons du livre, festivals"
10,1623,2024-01-21,Le corps fait son cinéma !,Espace RDC,Atelier,Tout public,None,28.0,NaN,Nuits de la lecture,None,Tous publics,1,"Fêtes, salons du livre, festivals"
11,1624,2024-01-21,Corps accords,La criée,Lecture/contes,Tout public,None,25.0,NaN,Nuits de la lecture,None,Tous publics,1,"Fêtes, salons du livre, festivals"
18,1631,2024-02-02,Questions de parents : épuisement parental,La criée,Rencontre/conférence,Adultes,None,7.0,NaN,Week-end des bébés,None,Tous publics,1,"Fêtes, salons du livre, festivals"
23,1636,2024-02-03,À la pêche aux albums !,Salle d'heure du conte,Lecture/contes,Enfants,None,28.0,NaN,Week-end des bébés,14 enfants et 14 adultes,Enfants,1,"Fêtes, salons du livre, festivals"
25,1638,2024-02-03,Veillée musicale,Espace Jeunesse,Concert,Enfants,None,65.0,NaN,Week-end des bébés,None,Enfants,1,"Fêtes, salons du livre, festivals"
26,1639,2024-02-04,Racontée musicale spéciale,Salle d'heure du conte,Lecture/contes,Enfants,None,110.0,NaN,Week-end des bébés,None,Enfants,1,"Fêtes, salons du livre, festivals"


### H428 - Autres

In [127]:
ac_2023.loc[(ac_2023['type']=='Autre',['type_sll'])] = 'Autre'

In [128]:
len(ac_2023[(ac_2023['type']=='Autre')&
        (ac_2023['public']=='Tout public')
       ]
   )

5

In [129]:
len(ac_2023[(ac_2023['type']=='Autre')&
        (ac_2023['public']=='Enfants')
       ]
   )

1

In [130]:
ac_2023[ac_2023['type_sll']=='Autre']

,id,date,action,lieu,type,public,partenariat,participants,jauge,evenement,notes,public_sll,nb_actions,type_sll
52,1665,2024-03-23,Speed-dating patrimonial,Magasins,Autre,Adultes,None,14.0,NaN,None,None,Tous publics,1,Autre
54,1667,2024-03-23,Dictée,La criée,Autre,Adultes,None,41.0,NaN,Semaine Langue française & francophonie,None,Tous publics,1,Autre
79,1692,2024-05-12,Vide dressing,La criée,Autre,Tout public,Do it Céline Caffè,35.0,NaN,None,None,Tous publics,1,Autre
103,1721,2024-06-08,Trésors du Patrimoine et des Archives,La criée,Autre,Tout public,None,8.0,NaN,À la découverte des manuscrits,None,Tous publics,1,Autre
105,1723,2024-06-09,Trésors du Patrimoine et des Archives,La criée,Autre,Tout public,None,6.0,NaN,À la découverte des manuscrits,None,Tous publics,1,Autre
139,1852,2024-09-28,Grande braderie de La Grand-Plage,Espace RDC,Autre,Tout public,None,0.0,NaN,None,None,Tous publics,1,Autre
140,1853,2024-09-29,Vide dressing,Espace RDC,Autre,Tout public,Do It Céline Caffè,60.0,50.0,None,None,Tous publics,1,Autre
158,1871,2024-10-12,Visites,Toute la médiathèque,Autre,Adultes,None,32.0,30.0,Nuit des bibliothèques,None,Tous publics,1,Autre
167,1880,2024-10-19,Graines de philo enfant,Salle d'heure du conte,Autre,Enfants,None,15.0,15.0,None,None,Enfants,1,Autre
176,1889,2024-11-10,Karaoké en famille,La criée,Autre,Familles,None,64.0,40.0,None,None,Tous publics,1,Autre


### Formations au public

In [131]:
ac_2023[ac_2023['type']=='Atelier']

,id,date,action,lieu,type,public,partenariat,participants,jauge,evenement,notes,public_sll,nb_actions,type_sll
3,1616,2024-01-14,Tricot des Lulus,La criée,Atelier,Tout public,None,22.0,20.0,None,None,Tous publics,1,NaN
4,1617,2024-01-16,Atelier informatique pour débutants,Espace multimédia,Atelier,Adultes,None,10.0,12.0,None,None,Tous publics,1,NaN
5,1618,2024-01-19,Radio à La Grand-Plage,Salle VDM,Atelier,Adultes,None,15.0,10.0,None,None,Tous publics,1,NaN
10,1623,2024-01-21,Le corps fait son cinéma !,Espace RDC,Atelier,Tout public,None,28.0,NaN,Nuits de la lecture,None,Tous publics,1,"Fêtes, salons du livre, festivals"
12,1625,2024-01-24,Dans tes rêves,La criée,Atelier,Enfants,La Condition Publique,45.0,NaN,None,None,Enfants,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1912,2024-12-06,Radio à La Grand-Plage,Salle VDM,Atelier,Adultes,None,10.0,15.0,None,None,Tous publics,1,NaN
206,1919,2024-12-13,Radio à La Grand-Plage,Salle VDM,Atelier,Adultes,None,8.0,15.0,None,None,Tous publics,1,NaN
211,1924,2024-12-15,Mon maquillage Zéro Déchet pour les fêtes !,La criée,Atelier,Tout public,None,18.0,20.0,None,None,Tous publics,1,NaN
213,1926,2024-12-20,Radio à La Grand-Plage,Salle VDM,Atelier,Adultes,None,4.0,15.0,None,None,Tous publics,1,NaN


In [132]:
ac_2023.pivot_table(index=['type_sll','public_sll'],
                    values='nb_actions',
                    aggfunc=sum
                   )

nb_actions
type_sll                          public_sll              
Autre                             Enfants                1
                                  Tous publics          10
Club lecture                      Tous publics           6
Concerts, projections             Enfants                6
                                  Tous publics          13
Conférences, rencontres, lectures Tous publics          24
Exposition                        Tous publics           1
Fêtes, salons du livre, festivals Enfants               12
                                  Tous publics          34
Séances de conte                  Enfants                9
                                  Tous publics           3

In [133]:
ac_2023.pivot_table(index='type_sll',
                    values='participants',
                    aggfunc=sum
                   )

,participants
type_sll,
Autre,279.0
Club lecture,72.0
"Concerts, projections",720.0
"Conférences, rencontres, lectures",555.0
Exposition,18.0
"Fêtes, salons du livre, festivals",1628.0
Séances de conte,417.0


# K - Patrimoine

## K2 - Fonds et acquisitions

In [134]:
query = f"""SELECT
                    i.itemnumber,
                    i.barcode,
                    i.dateaccessioned,
                    i.price,
                    i.homebranch,
                    i.holdingbranch,
                    i.location,
                    i.ccode,
                    i.itemcallnumber,
                    i.notforloan,
                    i.damaged,
                    DATE(i.damaged_on),
                    i.withdrawn,
                    DATE(i.withdrawn_on),
                    i.itemlost,
                    DATE(i.itemlost_on),
                    i.onloan,
                    i.datelastborrowed,
                    i.biblionumber,
                    b.title as titre,
                    bi.publicationyear,
                    bi.itemtype,
                    i.timestamp
                FROM koha{year}.items i
                JOIN koha{year}.biblioitems bi ON i.biblionumber = bi.biblionumber
                JOIN koha{year}.biblio b ON i.biblionumber = b.biblionumber
                WHERE i.notforloan IN (-1, -2, 0, 2, -4, 5)
                AND i.homebranch != 'MUS'"""
items = pd.read_sql(query, db_conn)

In [135]:
exemplaires = Document(df=items, db_conn=db_conn)
exemplaires.get_doc_statdb_data()
exemplaires.get_doc_es_data()

In [136]:
pat_ccode = ['PENACZZ', 'PENCVZZ', 'PENDEZZ', 'PENHPZZ', 'PENPDZZ', 'PENRSZZ', 'AAPPRLP',
             'PPAFIZZ', 'PPEFGZZ', 'PPELGZZ', 'PPEPMZZ', 'PPEPRZZ', 'PPIPIZZ', 'P17']
patrimoine_df = exemplaires.df[exemplaires.df['doc_item_collection_ccode'].isin(pat_ccode)]
patrimoine_df['sll_acq'] = 'non'
patrimoine_df.loc[patrimoine_df['doc_item_date_creation'].astype('str').str[0:4] == '2022', 'sll_acq' ] = 'oui'

/tmp/ipykernel_2550/2479472917.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patrimoine_df['sll_acq'] = 'non'


In [137]:
len(patrimoine_df)

62528

### Livres imprimés

In [138]:
data = patrimoine_df[patrimoine_df['sll_collection'] == 'D1 - Livres imprimés']

In [139]:
len(data)

35947

In [140]:
len(data[data['sll_acq'] == 'oui'])

444

In [141]:
data['publicationyear_'] = data['publicationyear'].str.extract(r'(^\d{4})').astype(float)
l1811 = data[data['publicationyear_'] < 1811]
len(l1811)

/tmp/ipykernel_2550/2076920371.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['publicationyear_'] = data['publicationyear'].str.extract(r'(^\d{4})').astype(float)


1055

In [142]:
len(l1811[l1811['sll_acq'] == 'oui'])

33

In [143]:
len(patrimoine_df[patrimoine_df['doc_item_collection_ccode'] == 'AAPPRLP'])

546

In [144]:
len(patrimoine_df[(patrimoine_df['doc_item_collection_ccode'] == 'AAPPRLP') & (patrimoine_df['sll_acq'] == 'oui')])

0

### Publications en séries

In [145]:
data = patrimoine_df[patrimoine_df['sll_collection'] == 'D1 - Publications en série imprimées']
len(data)

1362

In [146]:
len(data[data['sll_acq'] == 'oui'])

5

### Cartes

In [147]:
data = patrimoine_df[patrimoine_df['sll_collection'] == 'D3 – Documents cartographiques']
print(f"Fonds : {len(data)}")
print(f"Acquisitions : {len(data[data['sll_acq'] == 'oui'])}")

Fonds : 0
Acquisitions : 0


### Partitions

In [148]:
data = patrimoine_df[patrimoine_df['sll_collection'] == 'D3 – Musique imprimée']
print(f"Fonds : {len(data)}")
print(f"Acquisitions : {len(data[data['sll_acq'] == 'oui'])}")

Fonds : 1661
Acquisitions : 90


### Documents iconographiques

In [149]:
data = patrimoine_df[patrimoine_df['sll_collection'] == 'D3 - Documents graphiques']
print(f"Fonds : {len(data)}")
print(f"Acquisitions : {len(data[data['sll_acq'] == 'oui'])}")

Fonds : 94
Acquisitions : 0


### Documents sonores

In [150]:
data = patrimoine_df[patrimoine_df['sll_collection'].isin(['D4 - Documents audiovisuels fonds adultes / Documents sonores : musique',
                                                           'D4 - Documents audiovisuels fonds adultes / Documents sonores : livres enregistrés'])]
print(f"Fonds : {len(data)}")
print(f"Acquisitions : {len(data[data['sll_acq'] == 'oui'])}")

Fonds : 22538
Acquisitions : 157


### Images animées

In [151]:
data = patrimoine_df[patrimoine_df['sll_collection'].isin(['D4 - Documents audiovisuels fonds adultes / documents vidéo adultes',
                                                           'D4 - Documents audiovisuels fonds enfants / documents vidéo enfants'])]
print(f"Fonds : {len(data)}")
print(f"Acquisitions : {len(data[data['sll_acq'] == 'oui'])}")

Fonds : 895
Acquisitions : 0


D1 - Livres imprimés                                                                  205053
D4 - Documents audiovisuels fonds adultes / Documents sonores : musique                45233
D4 - Documents audiovisuels fonds adultes / documents vidéo adultes                    12067
D1 - Publications en série imprimées                                                    9173
D3 – Musique imprimée                                                                   3193
D4 - Documents audiovisuels fonds enfants / documents vidéo enfants                     1593
D4 - Documents audiovisuels fonds adultes / Documents sonores : livres enregistrés      1541
D4 - Documents audiovisuels fonds enfants / Documents sonores : musique                  901
D4 - Total documents multimédia sur support                                              287
D4 - Documents audiovisuels fonds enfants / Documents sonores : livres enregistrés       218
D3 – Documents cartographiques                                                           125
D3 - Documents graphiques                                                                 93
D3 - Autres documents                                                                     83